In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ast import literal_eval
from pathlib import Path

In [3]:
from ecephys.scoring import visbrain_hypnogram_to_datetime, write_datetime_hypnogram
from ecephys.sglx_utils import load_timeseries
import ecephys_analyses as ea

In [4]:
def load_sr_chans(sr_chans_path):
    df = pd.read_csv(sr_chans_path)
    df.sr_chans = df.sr_chans.apply(lambda x: [] if pd.isnull(x) else list(literal_eval(x)))
    df["duration"] = df.apply(lambda row: row.end_time - row.start_time, axis=1)
    
    return df

In [5]:
def write_datetime_format(sr_chans_dt, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    sr_chans_dt.to_csv(
        path,
        columns=["sr_chans", "start_time", "end_time", "duration"],
        sep="\t",
        index=False
    )

In [6]:
def write_experiment(subject, experiment):
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.csv")
    sr_chans_dt_paths = ea.get_sglx_style_datapaths(subject, experiment, "all", ext="sr_chans.tsv")
    
    all_sr_chans = [load_sr_chans(path) for path in sr_chans_paths]
    sigs = [load_timeseries(path, chans=[0], start_time=0, end_time=1) for path in bin_paths]
        
    for sig, sr_chans, sr_chans_dt_path in zip(sigs, all_sr_chans, sr_chans_dt_paths):
        file_start_dt = pd.to_datetime(sig.fileCreateTime)
        sr_chans_dt = visbrain_hypnogram_to_datetime(sr_chans, file_start_dt)
        write_datetime_format(sr_chans_dt, sr_chans_dt_path)

In [14]:
write_experiment("Allan", "ketamine")

nChan: 385, nFileSamp: 17610153
nChan: 385, nFileSamp: 17999996
nChan: 385, nFileSamp: 17999996
nChan: 385, nFileSamp: 8530138
